In [1]:
import os
import subprocess
import gdal
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import random 
%matplotlib inline

In [2]:
city="ams"
base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath("__file__")))))
print(base_dir)
# Paths for the Population Data --------------------------------------------------------------
#path to ancillary data folder
ancillary_data_folder_path = base_dir + "/data_prep/{}_Projectdata/AncillaryData".format(city)
ancillary_POPdata_folder_path = base_dir + "/data_prep/{}_Projectdata/PopData".format(city)
image_path= base_dir + "/data_prep/{}_Projectdata/PopData/images".format(city)
gdal_rasterize_path = r'C:/Users/NM12LQ/Anaconda3/envs/popnet_env/Library/bin'
years_list=[1992,1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2018] #

c:\FUME\PopNetV2


In [3]:
src_file = ancillary_POPdata_folder_path + "/Neighborhood&districtData/demographicProcess/neighDemo_2019.geojson"
polydf = gpd.read_file(src_file, crs="EPSG:3035")

In [ ]:
print(polydf.columns.tolist())

In [4]:
polydf = polydf[[ "Buurtcombinatie_code", "Buurtcombinatie", "coding" ,'geometry', "Number of inhabitants",  "total households" , "single households" , "households without children ",  "households with children" , "average household size", "age 0 to 15 years",  "age 15 to 25 years",  "age 45 to 65 years",  "a_45_64",  "age 65 years or older" ]]
polydf = polydf.set_index("coding")
polydf.head(2)

,Buurtcombinatie_code,Buurtcombinatie,geometry,Number of inhabitants,total households,single households,households without children,households with children,average household size,age 0 to 15 years,age 15 to 25 years,age 45 to 65 years,a_45_64,age 65 years or older
coding,,,,,,,,,,,,,,
WK036300,A00,Burgwallen-Oude Zijde,"POLYGON ((3974234.330 3264048.104, 3974216.369...",4295,3040,2100,690,250,1.4,235,580,2150,935,405
WK036301,A01,Burgwallen-Nieuwe Zijde,"POLYGON ((3974475.547 3264416.278, 3974448.698...",4120,2915,1990,720,200,1.4,215,590,2225,785,320


In [5]:
grid_df = gpd.read_file(ancillary_POPdata_folder_path + '/2018/temp_shp/2018_dataVectorGrid.geojson')
grid_df.head(2)

,grid_id,l1_totalpop,l2_children,l3_students,l4_mobile_adults,l5_not_mobile_adults,l6_elderly,l7_immigrants,l8_eu_immigrants,l9_noneu_immigrants,...,Northern_America,Latin_America_and_the_Caribbean,Africa,Northern_Africa,Sub-Saharan_Africa,OutsideEurope,Others,Colonies,totalMig,geometry
0,3455,1,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,"MULTIPOLYGON (((3963800.000 3267600.000, 39639..."
1,3456,4,1,1,0,2,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,"MULTIPOLYGON (((3963800.000 3267500.000, 39639..."


In [ ]:
grid_df.info

In [6]:
grid_df = grid_df[[ "grid_id",	"l1_totalpop",	"l2_children",	"l3_students",	"l4_mobile_adults",	"l5_not_mobile_adults",	"l6_elderly",'geometry']]

In [7]:
from geopandas.tools import sjoin
join_left_df = sjoin(grid_df, polydf, how="left")
join_left_df.head(2)

,grid_id,l1_totalpop,l2_children,l3_students,l4_mobile_adults,l5_not_mobile_adults,l6_elderly,geometry,index_right,Buurtcombinatie_code,...,total households,single households,households without children,households with children,average household size,age 0 to 15 years,age 15 to 25 years,age 45 to 65 years,a_45_64,age 65 years or older
0,3455,1,0,0,0,0,1,"MULTIPOLYGON (((3963800.000 3267600.000, 39639...",WK036310,B10,...,135.0,110.0,5.0,10.0,1.3,10.0,20.0,70.0,70.0,20.0
1,3456,4,1,1,0,2,0,"MULTIPOLYGON (((3963800.000 3267500.000, 39639...",WK036310,B10,...,135.0,110.0,5.0,10.0,1.3,10.0,20.0,70.0,70.0,20.0


In [41]:
frame.to_file(ancillary_POPdata_folder_path + "/Neighborhood&districtData/demographicProcess/join3.geojson",driver='GeoJSON',crs="EPSG:3035")

In [20]:
selectedColumns = join_left_df[['grid_id','geometry', "l1_totalpop","Number of inhabitants",  "total households" , "single households" , "households without children ",  "households with children" , "average household size" , "Buurtcombinatie_code", "l2_children",	"l3_students",	"l4_mobile_adults","l5_not_mobile_adults",	"l6_elderly", 'age 0 to 15 years', 'age 15 to 25 years', 'age 45 to 65 years', 'a_45_64', 'age 65 years or older' ]]
frame = selectedColumns.copy()

In [21]:
frame = frame.drop_duplicates(subset=['grid_id'], keep='first')


In [27]:
frame.dtypes

grid_id                            int64
geometry                        geometry
l1_totalpop                        int64
Number of inhabitants            float64
total households                 float64
single households                float64
households without children      float64
households with children         float64
average household size           float64
Buurtcombinatie_code              object
l2_children                        int64
l3_students                        int64
l4_mobile_adults                   int64
l5_not_mobile_adults               int64
l6_elderly                         int64
age 0 to 15 years                float64
age 15 to 25 years               float64
age 45 to 65 years               float64
a_45_64                          float64
age 65 years or older            float64
HouseholdsCell                   float64
ChildrenPerHousehold             float64
dtype: object

In [ ]:
frame.iloc[60:70 ]

In [23]:
frame["HouseholdsCell"] = frame["l1_totalpop" ] / frame["average household size"]

In [24]:
frame["ChildrenPerHousehold"] = frame["l2_children" ] / frame["HouseholdsCell"]

In [33]:
frame["HouseholdWithChildren"] = (frame["households with children" ] * frame["l1_totalpop"]/frame["Number of inhabitants"]) * ( (frame["age 45 to 65 years"] + frame[ "a_45_64"]) / ( frame["l4_mobile_adults"] + frame[ "l5_not_mobile_adults"]) ) * (frame["l2_children"] / frame[ "age 0 to 15 years" ])

In [42]:
frame["var"] = (frame["Number of inhabitants" ] - (frame["average household size" ] * frame["single households" ] ) - (frame["average household size" ] * frame["households without children " ])) / frame["households with children" ]

In [ ]:
frame["varB3"] = (frame["Number of inhabitants" ] - (frame["average household size" ] * frame["single households" ] ) - (frame["average household size" ] * frame["households without children " ])) / frame["households with children" ]

In [ ]:
total = frame.loc[frame["Buurtcombinatie_code"] == "A0", "HouseholdWithChildrenA"].sum() 
total

In [ ]:
frame["avHouseholdSize"] = frame["Number of inhabitants" ] / frame["total households"]

In [36]:
total = frame.loc[frame["Buurtcombinatie_code"] == "A1", "HouseholdWithChildren"].sum() 
total

0.0

In [43]:
frame

,grid_id,geometry,l1_totalpop,Number of inhabitants,total households,single households,households without children,households with children,average household size,Buurtcombinatie_code,...,age 15 to 25 years,age 45 to 65 years,a_45_64,age 65 years or older,HouseholdsCell,ChildrenPerHousehold,HouseholdWithChildren,HouseholdWithChildrenA,HouseholdWithChildrenB,var
0,3455,"MULTIPOLYGON (((3963800.000 3267600.000, 39639...",1,175.0,135.0,110.0,5.0,10.0,1.3,B10,...,20.0,70.0,70.0,20.0,0.769231,0.000,NaN,0.057143,0.000000,2.550000
1,3456,"MULTIPOLYGON (((3963800.000 3267500.000, 39639...",4,175.0,135.0,110.0,5.0,10.0,1.3,B10,...,20.0,70.0,70.0,20.0,3.076923,0.325,1.6,0.228571,0.022857,2.550000
2,4439,"MULTIPOLYGON (((3964300.000 3264200.000, 39644...",2,1125.0,395.0,155.0,110.0,140.0,2.2,F80,...,145.0,320.0,325.0,170.0,0.909091,0.000,NaN,0.248889,0.000000,3.871429
3,4617,"MULTIPOLYGON (((3964400.000 3265400.000, 39645...",4,2445.0,1035.0,315.0,310.0,420.0,2.4,F79,...,265.0,640.0,730.0,390.0,1.666667,0.000,0.0,0.687117,0.000000,2.250000
4,4618,"MULTIPOLYGON (((3964400.000 3265300.000, 39645...",5,2445.0,1035.0,315.0,310.0,420.0,2.4,F79,...,265.0,640.0,730.0,390.0,2.083333,0.000,0.0,0.858896,0.000000,2.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7506,40317,"MULTIPOLYGON (((3983200.000 3267400.000, 39833...",5,1250.0,445.0,50.0,165.0,230.0,2.8,N73,...,165.0,205.0,445.0,205.0,1.785714,1.680,3.9,0.920000,0.012000,2.817391
7507,40703,"MULTIPOLYGON (((3983400.000 3266800.000, 39835...",2,1250.0,445.0,50.0,165.0,230.0,2.8,N73,...,165.0,205.0,445.0,205.0,0.714286,0.000,0.0,0.368000,0.000000,2.817391
7508,41647,"MULTIPOLYGON (((3983900.000 3267400.000, 39840...",1,1250.0,445.0,50.0,165.0,230.0,2.8,N73,...,165.0,205.0,445.0,205.0,0.357143,0.000,NaN,0.184000,0.000000,2.817391
7509,42975,"MULTIPOLYGON (((3984600.000 3267600.000, 39847...",3,1250.0,445.0,50.0,165.0,230.0,2.8,N73,...,165.0,205.0,445.0,205.0,1.071429,0.000,0.0,0.552000,0.000000,2.817391


In [17]:
frame.dtypes

grid_id                            int64
geometry                        geometry
l1_totalpop                        int64
Number of inhabitants            float64
total households                 float64
single households                float64
households without children      float64
households with children         float64
average household size           float64
Buurtcombinatie_code              object
l2_children                        int64
l3_students                        int64
l4_mobile_adults                   int64
l5_not_mobile_adults               int64
l6_elderly                         int64
age 0 to 15 years                float64
age 15 to 25 years               float64
age 45 to 65 years               float64
a_45_64                          float64
age 65 years or older            float64
HouseholdsCell                   float64
dtype: object

In [ ]:
frame["avHouseholdSize"]

In [ ]:
ndf = pd.DataFrame()
ndf["Buurtcombinatie_code"] = frame['Buurtcombinatie_code']
ndf["ori"] = frame.groupby(['Buurtcombinatie_code'])['total households'].agg('sum')
ndf["cell"] = frame.groupby(['Buurtcombinatie_code'])['HouseholdsTotal'].agg('sum')
ndf["error"] = (ndf["ori"] - ndf["cell"]) / ndf["ori"].sum()

In [ ]:
ndf.head(5)

In [ ]:
frame["ERROR"] = frame["l1_totalpop" ] / frame["avHouseholdSize"]

In [38]:
frame["HouseholdWithChildren"] = (frame["households with children" ]*frame["l1_totalpop"]/frame["Number of inhabitants"]) * ( (frame["age 45 to 65 years"] + frame[ "a_45_64"]) / ( frame["l4_mobile_adults"] + frame[ "l5_not_mobile_adults"]) ) * (frame["l2_children"] / frame[ "age 0 to 15 years" ])

In [39]:
frame["HouseholdWithChildrenA"] = (frame["households with children" ]*frame["l1_totalpop"]/frame["Number of inhabitants"]) 

In [40]:
frame["HouseholdWithChildrenB"] = (frame["households with children" ] * frame["l1_totalpop"]/frame["Number of inhabitants"]) * (frame["l2_children"] / frame[ "age 0 to 15 years" ])

In [ ]:
print(frame["Buurtcombinatie_code"], frame["households with children"])

In [ ]:
print(frame["HouseholdWithChildren"].groupby("Buurtcombinatie").sum())

In [ ]:
frame["HouseholdWithoutChildren"] = (join_left_df["households without children " ]*join_left_df["l1_totalpop"]/join_left_df["Number of inhabitants"]) * ( (join_left_df["Number of inhabitants"] - join_left_df[ "age 0 to 15 years"]) / ( join_left_df["l1_totalpop"] - join_left_df[ "l2_children"]) ) 

In [ ]:
frame["HouseholdSingle"] = (join_left_df["single households" ]*join_left_df["l1_totalpop"]/join_left_df["Number of inhabitants"]) * ( (join_left_df["Number of inhabitants"] - join_left_df[ "age 0 to 15 years"]) / ( join_left_df["l1_totalpop"] - join_left_df[ "l2_children"]) ) 

In [ ]:
frame["totalHouseholdsCell"] = frame["HouseholdWithChildren"] + frame["HouseholdWithoutChildren"] + frame["HouseholdSingle"]

In [ ]:
frame["men"] = (join_left_df["men"]/join_left_df["Number of inhabitants"])*join_left_df["l1_totalpop"]
frame["women"] = (join_left_df["women"]/join_left_df["Number of inhabitants"])*join_left_df["l1_totalpop"]

frame["unmarried"] = (join_left_df["unmarried"]/join_left_df["Number of inhabitants"])*join_left_df["l1_totalpop"]
frame["married"] = (join_left_df["a_gehuwd"]/join_left_df["Number of inhabitants"])*join_left_df["l1_totalpop"]
frame["divorced"] = (join_left_df["divorced"]/join_left_df["Number of inhabitants"])*join_left_df["l1_totalpop"]
frame["widdowed"] = (join_left_df["widdowed"]/join_left_df["Number of inhabitants"])*join_left_df["l1_totalpop"]

In [ ]:
#list_of_columns_to_exclude = ['grid_id','geometry', "l1_totalpop" ]
#frame = frame[[i for i in list(frame.columns) if i not in [list_of_columns_to_exclude]]].round(1)
frame.to_file(ancillary_POPdata_folder_path + "/Neighborhood&districtData/demographicProcess/frame5.geojson",driver='GeoJSON',crs="EPSG:3035")

In [ ]:
def shptoraster(ancillary_POPdata_folder_path,ancillary_data_folder_path, gdal_rasterize_path, city):
    # Getting extent of ghs pop raster
    data = gdal.Open(os.path.dirname(ancillary_data_folder_path) + "/temp_tif/{0}_CLC_2012_2018.tif".format(city))
    wkt = data.GetProjection()
    geoTransform = data.GetGeoTransform()
    minx = geoTransform[0]
    maxy = geoTransform[3]
    maxx = minx + geoTransform[1] * data.RasterXSize
    miny = maxy + geoTransform[5] * data.RasterYSize
    data = None
    xres=100
    yres=100
    # Rasterizing layers
    src_file = ancillary_POPdata_folder_path + "/Neighborhood&districtData/demographicProcess/join.geojson"
    df = gpd.read_file(src_file)
    
    for column_name in df.columns:
        
        print("Rasterizing {} layer".format(column_name))
        dst_file = ancillary_POPdata_folder_path + "/Neighborhood&districtData/demographicProcess/{0}.tif".format( column_name)
        cmd = '{0}/gdal_rasterize.exe -a "{9}" -te {1} {2} {3} {4} -tr {5} {6} "{7}" "{8}"'\
            .format(gdal_rasterize_path, minx, miny, maxx, maxy, xres, yres, src_file, dst_file, column_name)
        print(cmd) #-ot Integer64
        subprocess.call(cmd, shell=True)

In [ ]:
gdal_rasterize_path = r'C:/Users/NM12LQ/Anaconda3/envs/popnet_env/Library/bin'

shptoraster(ancillary_POPdata_folder_path,ancillary_data_folder_path, gdal_rasterize_path, city)